In [3]:
# from langchain import PromptTemplate
# from langchain.chains import RetrievalQA
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.llms import CTransformers  # Capitalization might be needed depending on version
# # from langchain_community.vectorstores import Pinecone
# import pinecone


In [ ]:
def load_pdf(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    
    documents=loader.load()
    return documents

In [5]:
extracted_data=load_pdf("data/")

In [6]:
# external_data

In [7]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [8]:
text_chunks=text_split(extracted_data)
# print("length",len(text_chunks))

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings
def download_hugging_face_embedding():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embeddings=download_hugging_face_embedding()

C:\Users\deepa\AppData\Local\Temp\ipykernel_4520\876629269.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\deepa\OneDrive\Desktop\medical chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
query_result=embeddings.embed_query("hello")
print("length",len(query_result))

length 384


In [13]:
query_result

[-0.0627717599272728,
 0.05495879426598549,
 0.052164845168590546,
 0.0857900083065033,
 -0.0827488973736763,
 -0.07457297295331955,
 0.06855470687150955,
 0.018396394327282906,
 -0.08201129734516144,
 -0.03738484904170036,
 0.012124893255531788,
 0.0035183229483664036,
 -0.004134293645620346,
 -0.04378441721200943,
 0.021807299926877022,
 -0.00510272104293108,
 0.019546564668416977,
 -0.0423487089574337,
 -0.11035966873168945,
 0.005424527917057276,
 -0.055734783411026,
 0.02805238962173462,
 -0.02315872348845005,
 0.028481375426054,
 -0.05370955914258957,
 -0.052601538598537445,
 0.03393925353884697,
 0.045388635247945786,
 0.023718422278761864,
 -0.07312081754207611,
 0.05477774888277054,
 0.0170473363250494,
 0.08136039972305298,
 -0.0028627056162804365,
 0.011958062648773193,
 0.07355856150388718,
 -0.09423744678497314,
 -0.0813620463013649,
 0.04001539200544357,
 0.0006921913009136915,
 -0.01339334063231945,
 -0.05453811213374138,
 0.005151401273906231,
 -0.026139769703149796,
 0

In [14]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [15]:
PINECONE_API_KEY=os.environ.get("PINECONE_API_KEY")

In [16]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc=Pinecone(api_key=PINECONE_API_KEY)

index_name="medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-lud9ekx.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [17]:
import os 
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY

In [18]:
from langchain_pinecone import PineconeVectorStore

docsearch =PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [19]:
from langchain_pinecone import PineconeVectorStore

docsearch= PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [20]:
docsearch


In [21]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [22]:
retriever_docs=retriever.invoke("What is acne?")

In [23]:
retriever_docs

[Document(id='ed73a99c-986b-416a-9a28-74f3f077f227', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='d90e488e-9be0-48e2-8ef1-9585c50663ad', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM 

In [24]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [25]:
from langchain.prompts import PromptTemplate

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)


In [26]:
from langchain.llms import CTransformers
llm = CTransformers(
    model="model/llama-2-7b-chat.ggmlv3.q2_K.bin",
    model_type="llama",
    config={
        'max_new_tokens': 512,
        'temperature': 0.8
    }
)


In [27]:
from langchain.chains import RetrievalQA

qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
   
)

In [31]:
# while True:
#     user_input=input(f"Input Prompt:")
#     result=qa({'retriever_docs': user_input})
#     print("response:",result["result"])

In [ ]:
import time

while True:
    user_input = input("Input Prompt: ")
    print("Sending query...")
    start = time.time()
    result = qa.invoke({'query': user_input})
    end = time.time()
    print(f"Response received in {end - start:.2f} seconds")
    print("response:", result["result"])
